# **BWM: Modeling the Factors Influencing the Implementation of Violence Against Women (VAW) Laws in the Philippines Using Fuzzy Multi-Criteria Decision-Making**
**Author**: Daenielle Rai Peladas 
</br>
**Institution**: University of the Philippines - Cebu 
</br>
**Laboratory**: Computational Intelligence and Machine Learning 
</br>
**Advisor**: Associate Professior Dharyll Prince Abellana 
</br>
</br>

**Date Created**: May 17, 2025
</br>
**Date Modified**: May 17, 2025
</br>

This Jupyter notebook implements the Best-Worst Method (BWM) to prioritize broad categories influencing the factors affecting the implementation of Violence Against Women (VAW) laws in the Philippines.

The analysis is based on expert evaluations gathered through interviews, allowing for the identification and ranking of key factors by comparing the most and least important criteria. This prioritization highlights which aspects require the most attention to improve policy implementation effectiveness.

## **Imports and Setup**

In [10]:
# Import library
from pyDecision.algorithm import bw_method as bwm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

## **Helper Functions**

In [11]:
def run_bwm_for_expert(df, expert_id, num_criteria):
    """
    Run BWM algorithm for single expert
    """

    # Filter dataframe for the specific expert
    expert_df = df[df['Expert'] == expert_id]

    # Drop first column (expert_id)
    expert_df = expert_df.drop(columns=['Expert'])

    # Split dataset into two sets based on the number of criteria
    # Best to others 
    bo = np.array(expert_df.iloc[0, :num_criteria])
    # Others to worst
    ow = np.array(expert_df.iloc[0, num_criteria:])
    
    # Run model
    weights = bwm(bo, ow, eps_penalty=1)

    return {
        'Expert ID': expert_id,
        'Weights': weights
    }

In [12]:
def aggregate_expert_weights(expert_results, aggregation_method='mean'):
    """
    Aggregate weights from multiple experts
    """
    
    all_weights = np.array([result['Weights'] for result in expert_results])

    if aggregation_method == 'arithmetic_mean':
        return np.mean(all_weights, axis=0)
    elif aggregation_method == 'geometric_mean':
        return np.power(np.prod(all_weights, axis=0), 1/len(all_weights))
    else:
        raise ValueError("Unsupported aggregation method. Use 'arithmetic_mean' or 'geometric_mean'.")

In [13]:
def add_cr_psi_from_dict(expert_list, cr_psi_values):
    for expert in expert_list:
        expert_id = expert['Expert ID']
        if expert_id in cr_psi_values:
            expert['Consistency Ratio'] = cr_psi_values[expert_id]['CR']
            expert['PSI'] = cr_psi_values[expert_id]['PSI']
        else:
            print(f"No values provided for {expert_id}. Skipping.")

In [14]:
def get_best_experts_by_cr(expert_list, threshold=0.1):
    # Filter experts with CR below threshold
    best_experts = [e for e in expert_list if e.get('Consistency Ratio', 1) < threshold]
    
    if best_experts:
        return best_experts
    else:
        # Find minimum CR in the list
        min_cr = min(e.get('Consistency Ratio', 1) for e in expert_list)
        # Return all experts with the minimum CR (in case of ties)
        return [e for e in expert_list if e.get('Consistency Ratio', 1) == min_cr]

## **Factors Involved**

### **Systemic Issues**

In [15]:
# Import dataset
df_sys = pd.read_csv('bwm_dataset/bwm_sys.csv')
df_sys.head()

,Expert,BO. Slow justice system causing delayed cases,BO. Heavy bureaucratic systems,BO. Mediation practices,BO. Lack of legal escalation due to preference of counseling,BO. Duty-bearers requiring face-to-face confrontation between victim-survivor and perpetrator,BO. Inadequately trained staff,BO. Inadequately empowered staff,BO. Frequent personnel changes requiring repeated capacity building,BO. Informal partnerships between agencies hindering sustainable connections,BO. Barangay Protection Orders are not utilized or delayed,BO. Absence of standardized protocols within help desks,BO. Cases being passed around offices without proper attention and resolution,BO. Providing service only for the sake of compliance within help desks,BO. Lack of confidentiality measures within help desks,BO. Jurisdictional limitations and poor inter-barangay collaboration,OW. Slow justice system causing delayed cases,OW. Heavy bureaucratic systems,OW. Mediation practices,OW. Lack of legal escalation due to preference of counseling,OW. Duty-bearers requiring face-to-face confrontation between victim-survivor and perpetrator,OW. Inadequately trained staff,OW. Inadequately empowered staff,OW. Frequent personnel changes requiring repeated capacity building,OW. Informal partnerships between agencies hindering sustainable connections,OW. Barangay Protection Orders are not utilized or delayed,OW. Absence of standardized protocols within help desks,OW. Cases being passed around offices without proper attention and resolution,OW. Providing service only for the sake of compliance within help desks,OW. Lack of confidentiality measures within help desks,OW. Jurisdictional limitations and poor inter-barangay collaboration
0,Expert 1,1,5,6,6,5,2,3,7,2,8,6,2,7,7,6,8,5,5,7,7,6,6,7,5,7,6,8,7,1,6
1,Expert 2,7,5,5,6,6,4,4,5,5,4,1,5,5,4,4,6,6,6,6,6,6,1,7,6,6,7,6,7,6,6
2,Expert 3,6,6,6,6,3,3,3,1,6,6,6,7,5,8,6,6,6,6,6,3,5,5,5,6,6,3,1,3,3,3
3,Expert 4,7,3,5,5,5,9,7,1,8,6,1,5,5,2,3,7,3,7,7,1,7,7,5,3,8,7,3,5,7,7
4,Expert 5,1,3,3,7,3,1,1,1,3,5,1,5,1,5,1,5,5,3,5,3,5,5,9,1,3,8,3,5,3,5


In [16]:
sys_expert_results = []

for expert in df_sys['Expert'].unique():
    expert_result = run_bwm_for_expert(df_sys, expert, 15)
    print(expert_result)
    print()
    sys_expert_results.append(expert_result)

CR: 0.8571 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2182
{'Expert ID': 'Expert 1', 'Weights': array([0.14107144, 0.07108155, 0.0598761 , 0.05986654, 0.06887903,
       0.06964626, 0.06950119, 0.05132237, 0.07132258, 0.04490707,
       0.0598761 , 0.08389679, 0.05132237, 0.03758461, 0.05984601])}

CR: 0.8333 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1056
{'Expert ID': 'Expert 2', 'Weights': array([0.04375257, 0.06125361, 0.06125361, 0.0506604 , 0.05098027,
       0.06570218, 0.02375369, 0.06125361, 0.05793461, 0.07079448,
       0.20064141, 0.06125361, 0.06064921, 0.06503556, 0.06508118])}

CR: 0.5 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1422
{'Expert ID': 'Expert 3', 'Weights': array([0.05969221, 0.04902008, 0.05505952, 0.05920235, 0.06610815,
       0.06610819, 0.06639812, 0.21903541, 0.06043561, 0.05102233,
       0.04857894, 0.03181996, 0.06572932, 0.04184019, 0.05994962])}

CR: 0.75 (The Consistency Level is Not Acceptable)
Epsi

In [17]:
sys_cr_psi_values = {
    'Expert 1': {'CR': 0.8571, 'PSI': 0.2182},
    'Expert 2': {'CR': 0.8333, 'PSI': 0.1056},
    'Expert 3': {'CR': 0.5, 'PSI': 0.1422},
    'Expert 4': {'CR': 0.75, 'PSI': 0.2192},
    'Expert 5': {'CR': 0.6667, 'PSI': 0.1795},
    'Expert 6': {'CR': 0.875, 'PSI': 0.1551},
    'Expert 7': {'CR': 0.6667, 'PSI': 0.2314},
    'Expert 8': {'CR': 1.3333, 'PSI': 0.2187},
    'Expert 9': {'CR': 0.3611, 'PSI': 0.0489}
}

add_cr_psi_from_dict(sys_expert_results, sys_cr_psi_values)

In [18]:
sys_expert_results

[{'Expert ID': 'Expert 1',
  'Weights': array([0.14107144, 0.07108155, 0.0598761 , 0.05986654, 0.06887903,
         0.06964626, 0.06950119, 0.05132237, 0.07132258, 0.04490707,
         0.0598761 , 0.08389679, 0.05132237, 0.03758461, 0.05984601]),
  'Consistency Ratio': 0.8571,
  'PSI': 0.2182},
 {'Expert ID': 'Expert 2',
  'Weights': array([0.04375257, 0.06125361, 0.06125361, 0.0506604 , 0.05098027,
         0.06570218, 0.02375369, 0.06125361, 0.05793461, 0.07079448,
         0.20064141, 0.06125361, 0.06064921, 0.06503556, 0.06508118]),
  'Consistency Ratio': 0.8333,
  'PSI': 0.1056},
 {'Expert ID': 'Expert 3',
  'Weights': array([0.05969221, 0.04902008, 0.05505952, 0.05920235, 0.06610815,
         0.06610819, 0.06639812, 0.21903541, 0.06043561, 0.05102233,
         0.04857894, 0.03181996, 0.06572932, 0.04184019, 0.05994962]),
  'Consistency Ratio': 0.5,
  'PSI': 0.1422},
 {'Expert ID': 'Expert 4',
  'Weights': array([0.05163341, 0.0804209 , 0.07154891, 0.07108516, 0.03492874,
        

In [19]:
cr_pass = get_best_experts_by_cr(sys_expert_results, threshold=0.1)
cr_pass

[{'Expert ID': 'Expert 9',
  'Weights': array([0.13455929, 0.10221377, 0.06103667, 0.01223266, 0.01967587,
         0.0883882 , 0.08668246, 0.09645464, 0.02038701, 0.06109258,
         0.05861455, 0.10169643, 0.03669799, 0.06116332, 0.05910457]),
  'Consistency Ratio': 0.3611,
  'PSI': 0.0489}]

In [20]:
sys_aggregated_weights = aggregate_expert_weights(cr_pass, aggregation_method='arithmetic_mean')
sys_aggregated_weights

array([0.13455929, 0.10221377, 0.06103667, 0.01223266, 0.01967587,
       0.0883882 , 0.08668246, 0.09645464, 0.02038701, 0.06109258,
       0.05861455, 0.10169643, 0.03669799, 0.06116332, 0.05910457])

### **Resource Limitations**

In [21]:
# Import dataset
df_res =  pd.read_csv('bwm_dataset/bwm_res.csv')
df_res.head()

,Expert,BO. Lack of resources within barangay women's desk,BO. Underutilization of GAD budget,BO. No designated funding for barangays to handle VAWC cases,BO. Lack of financial resources for help desks and service providers,"BO. Unequal capacities of LGU to provide service, assistance, and support to victim-survivors and their families",BO. Overworked staff handling VAW cases,BO. Insufficient training for barangay VAWC focal persons,BO. Lack of psychosocial programming,BO. Lack of effective asset management mechanism,OW. Lack of resources within barangay women's desk,OW. Underutilization of GAD budget,OW. No designated funding for barangays to handle VAWC cases,OW. Lack of financial resources for help desks and service providers,"OW. Unequal capacities of LGU to provide service, assistance, and support to victim-survivors and their families",OW. Overworked staff handling VAW cases,OW. Insufficient training for barangay VAWC focal persons,OW. Lack of psychosocial programming,OW. Lack of effective asset management mechanism
0,Expert 1,7,1,7,7,6,5,8,7,7,3,7,1,3,2,2,7,6,6
1,Expert 2,8,6,7,1,8,8,8,8,7,7,1,6,7,7,6,7,7,7
2,Expert 3,7,1,4,4,5,7,7,7,5,6,6,1,6,7,7,6,7,7
3,Expert 4,8,1,6,8,6,3,6,5,8,7,7,6,1,4,5,7,7,8
4,Expert 5,1,1,5,1,1,1,5,5,1,1,1,1,1,5,7,6,3,7


In [22]:
res_expert_results = []

for expert in df_res['Expert'].unique():
    expert_result = run_bwm_for_expert(df_res, expert, 9)
    print(expert_result)
    print()
    res_expert_results.append(expert_result)

CR: 0.8571 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2248
{'Expert ID': 'Expert 1', 'Weights': array([0.08536236, 0.37774976, 0.04124366, 0.08038763, 0.10020757,
       0.07422504, 0.07107941, 0.08532995, 0.08441462])}

CR: 0.8571 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1898
{'Expert ID': 'Expert 2', 'Weights': array([0.07475614, 0.03769154, 0.08653833, 0.41596082, 0.07402812,
       0.07572149, 0.07492384, 0.07488371, 0.08549602])}

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1912
{'Expert ID': 'Expert 3', 'Weights': array([0.06811394, 0.34052567, 0.03735627, 0.11752552, 0.10584842,
       0.07579166, 0.0759194 , 0.07593696, 0.10298215])}

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1752
{'Expert ID': 'Expert 4', 'Weights': array([0.07068286, 0.3902924 , 0.07578761, 0.03073168, 0.09424382,
       0.08332564, 0.07427259, 0.10998052, 0.07068286])}

CR: 1.25 (The Consistency Level is Not Acceptable)
Epsilon

In [36]:
res_cr_psi_values = {
    'Expert 1': {'CR': 0.8571, 'PSI': 0.2248},
    'Expert 2': {'CR': 0.8571, 'PSI': 0.1898},
    'Expert 3': {'CR': 1.0, 'PSI': 0.1912},
    'Expert 4': {'CR': 1.0, 'PSI': 0.1752},
    'Expert 5': {'CR': 1.25, 'PSI': 0.318},
    'Expert 6': {'CR': 0.75, 'PSI': 0.1968},
    'Expert 7': {'CR': 0.6667, 'PSI': 0.1889},
    'Expert 8': {'CR': 0.7321, 'PSI': 0.1369},
    'Expert 9': {'CR': 0.75, 'PSI': 0.1192}
}

add_cr_psi_from_dict(res_expert_results, res_cr_psi_values)

In [37]:
res_expert_results

[{'Expert ID': 'Expert 1',
  'Weights': array([0.08536236, 0.37774976, 0.04124366, 0.08038763, 0.10020757,
         0.07422504, 0.07107941, 0.08532995, 0.08441462]),
  'Consistency Ratio': 0.8571,
  'PSI': 0.2248},
 {'Expert ID': 'Expert 2',
  'Weights': array([0.07475614, 0.03769154, 0.08653833, 0.41596082, 0.07402812,
         0.07572149, 0.07492384, 0.07488371, 0.08549602]),
  'Consistency Ratio': 0.8571,
  'PSI': 0.1898},
 {'Expert ID': 'Expert 3',
  'Weights': array([0.06811394, 0.34052567, 0.03735627, 0.11752552, 0.10584842,
         0.07579166, 0.0759194 , 0.07593696, 0.10298215]),
  'Consistency Ratio': 1.0,
  'PSI': 0.1912},
 {'Expert ID': 'Expert 4',
  'Weights': array([0.07068286, 0.3902924 , 0.07578761, 0.03073168, 0.09424382,
         0.08332564, 0.07427259, 0.10998052, 0.07068286]),
  'Consistency Ratio': 1.0,
  'PSI': 0.1752},
 {'Expert ID': 'Expert 5',
  'Weights': array([0.06310688, 0.17440269, 0.0762171 , 0.13777344, 0.11519958,
         0.12376956, 0.0762171 , 0.0762

In [38]:
res_cr_pass = get_best_experts_by_cr(res_expert_results, threshold=0.1)
res_cr_pass

[{'Expert ID': 'Expert 7',
  'Weights': array([0.09115281, 0.06511136, 0.26690622, 0.16829929, 0.08175097,
         0.05095655, 0.09107956, 0.08812832, 0.09661492]),
  'Consistency Ratio': 0.6667,
  'PSI': 0.1889}]

In [39]:
res_aggregated_weights = aggregate_expert_weights(res_cr_pass, aggregation_method='arithmetic_mean')
res_aggregated_weights

array([0.09115281, 0.06511136, 0.26690622, 0.16829929, 0.08175097,
       0.05095655, 0.09107956, 0.08812832, 0.09661492])

### **Data Monitoring Challenges**

In [24]:
# Import dataset
df_dat = pd.read_csv('bwm_dataset/bwm_dat.csv')
df_dat.head()

,Expert,BO. Lack of data harmonization across the interconnected referral system,"BO. Inconsistent data collection, reporting, information sharing practices",BO. Misclassification of cases,BO. Absence of database system,BO. Fragmented monitoring mechanisms across agencies,OW. Lack of data harmonization across the interconnected referral system,"OW. Inconsistent data collection, reporting, information sharing practices",OW. Misclassification of cases,OW. Absence of database system,OW. Fragmented monitoring mechanisms across agencies
0,Expert 1,3,2,1,2,7,1,2,7,2,7
1,Expert 2,6,6,7,1,7,1,8,7,8,7
2,Expert 3,1,3,5,3,8,7,2,3,1,5
3,Expert 4,1,8,3,8,9,8,5,5,1,8
4,Expert 5,1,1,1,1,7,9,9,9,9,1


In [25]:
dat_expert_results = []

for expert in df_dat['Expert'].unique():
    expert_result = run_bwm_for_expert(df_dat, expert, 5)
    print(expert_result)
    print()
    dat_expert_results.append(expert_result)

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.232
{'Expert ID': 'Expert 1', 'Weights': array([0.04478642, 0.27144987, 0.3666757 , 0.23188058, 0.08520743])}

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2668
{'Expert ID': 'Expert 2', 'Weights': array([0.05082113, 0.1397581 , 0.11979265, 0.57173775, 0.11789038])}

CR: 0.5714 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2544
{'Expert ID': 'Expert 3', 'Weights': array([0.46063349, 0.23823599, 0.14300263, 0.06875126, 0.08937664])}

CR: 0.875 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2246
{'Expert ID': 'Expert 4', 'Weights': array([0.53340906, 0.09469344, 0.24907305, 0.03860197, 0.08422248])}

CR: 0.0476 (The Consistency Level is Acceptable)
Epsilon Value: 0.0288
{'Expert ID': 'Expert 5', 'Weights': array([0.23049505, 0.25699267, 0.25260089, 0.23109952, 0.02881187])}

CR: 0.5 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2715
{'Expert ID': 'Expert 6', 'Wei

In [40]:
dat_cr_psi_values = {
    'Expert 1': {'CR': 1.0, 'PSI': 0.232},
    'Expert 2': {'CR': 1.0, 'PSI': 0.2668},
    'Expert 3': {'CR': 0.5714, 'PSI': 0.2544},
    'Expert 4': {'CR': 0.875, 'PSI': 0.2246},
    'Expert 5': {'CR': 0.047, 'PSI': 0.0288},
    'Expert 6': {'CR': 0.5, 'PSI': 0.2715},
    'Expert 7': {'CR': 1.0, 'PSI': 0.2585},
    'Expert 8': {'CR': 0.75, 'PSI': 0.1926},
    'Expert 9': {'CR': 0.0833, 'PSI': 0.0432}
}

add_cr_psi_from_dict(dat_expert_results, dat_cr_psi_values)
dat_expert_results

[{'Expert ID': 'Expert 1',
  'Weights': array([0.04478642, 0.27144987, 0.3666757 , 0.23188058, 0.08520743]),
  'Consistency Ratio': 1.0,
  'PSI': 0.232},
 {'Expert ID': 'Expert 2',
  'Weights': array([0.05082113, 0.1397581 , 0.11979265, 0.57173775, 0.11789038]),
  'Consistency Ratio': 1.0,
  'PSI': 0.2668},
 {'Expert ID': 'Expert 3',
  'Weights': array([0.46063349, 0.23823599, 0.14300263, 0.06875126, 0.08937664]),
  'Consistency Ratio': 0.5714,
  'PSI': 0.2544},
 {'Expert ID': 'Expert 4',
  'Weights': array([0.53340906, 0.09469344, 0.24907305, 0.03860197, 0.08422248]),
  'Consistency Ratio': 0.875,
  'PSI': 0.2246},
 {'Expert ID': 'Expert 5',
  'Weights': array([0.23049505, 0.25699267, 0.25260089, 0.23109952, 0.02881187]),
  'Consistency Ratio': 0.047,
  'PSI': 0.0288},
 {'Expert ID': 'Expert 6',
  'Weights': array([0.48480023, 0.08403203, 0.25202253, 0.07110403, 0.10804119]),
  'Consistency Ratio': 0.5,
  'PSI': 0.2715},
 {'Expert ID': 'Expert 7',
  'Weights': array([0.11163397, 0.522

In [41]:
dat_cr_pass = get_best_experts_by_cr(dat_expert_results, threshold=0.1)
dat_cr_pass

[{'Expert ID': 'Expert 5',
  'Weights': array([0.23049505, 0.25699267, 0.25260089, 0.23109952, 0.02881187]),
  'Consistency Ratio': 0.047,
  'PSI': 0.0288},
 {'Expert ID': 'Expert 9',
  'Weights': array([0.03778956, 0.0701806 , 0.307715  , 0.31461573, 0.26969911]),
  'Consistency Ratio': 0.0833,
  'PSI': 0.0432}]

In [42]:
dat_aggregated_weights = aggregate_expert_weights(dat_cr_pass, aggregation_method='arithmetic_mean')
dat_aggregated_weights

array([0.1341423 , 0.16358664, 0.28015794, 0.27285763, 0.14925549])

### **Sociocultural Dynamics**

In [27]:
# Import dataset
df_soc = pd.read_csv('bwm_dataset/bwm_soc.csv')
df_soc.head()

,Expert,BO. Patriarchal norms and gender biases,"BO. Power imbalances and bias based on class, race, or nationality",BO. Persistent belief in gender-based legal inequality,BO. Culture of victim-blaming,BO. Shaming victim-survivors,BO. Social stigma towards victims,BO. Fear of disrupting family relations,BO. Strong cultural emphasis on family obligations,BO. Underreporting of cases due to culture of silence,BO. Undervaluation of women's work outside the home,BO. Gossip culture,BO. Normalized cycle of abuse,BO. Emotional attachment to attacker-partner,"BO. Fear for their child's well-being, if involved",OW. Patriarchal norms and gender biases,"OW. Power imbalances and bias based on class, race, or nationality",OW. Persistent belief in gender-based legal inequality,OW. Culture of victim-blaming,OW. Shaming victim-survivors,OW. Social stigma towards victims,OW. Fear of disrupting family relations,OW. Strong cultural emphasis on family obligations,OW. Underreporting of cases due to culture of silence,OW. Undervaluation of women's work outside the home,OW. Gossip culture,OW. Normalized cycle of abuse,OW. Emotional attachment to attacker-partner,"OW. Fear for their child's well-being, if involved"
0,Expert 1,5,7,6,8,8,8,1,6,8,8,2,8,8,8,5,8,7,8,8,8,8,7,8,8,1,8,8,8
1,Expert 2,7,6,6,7,7,7,6,6,1,6,7,7,7,7,7,1,7,7,7,7,7,7,7,6,7,7,7,7
2,Expert 3,1,7,3,6,6,6,5,3,3,5,3,9,9,7,5,5,1,3,3,3,6,5,3,3,4,9,9,7
3,Expert 4,7,7,3,5,7,7,7,1,7,3,3,5,7,9,8,8,1,5,5,6,5,5,7,8,3,5,5,5
4,Expert 5,5,1,7,1,1,1,1,1,1,5,7,1,1,1,5,5,1,5,5,5,5,5,5,5,7,5,5,5


In [28]:
soc_expert_results = []

for expert in df_soc['Expert'].unique():
    expert_result = run_bwm_for_expert(df_soc, expert, 14)
    print(expert_result)
    print()
    soc_expert_results.append(expert_result)

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1988
{'Expert ID': 'Expert 1', 'Weights': array([0.07175668, 0.06421034, 0.07571554, 0.05734989, 0.05735006,
       0.05638111, 0.25998835, 0.0717893 , 0.05079942, 0.0482347 ,
       0.03070835, 0.04782889, 0.05522773, 0.05265963])}

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1836
{'Expert ID': 'Expert 2', 'Weights': array([0.06116048, 0.03225975, 0.07205954, 0.05049694, 0.05932126,
       0.06030516, 0.06848955, 0.0710692 , 0.24485596, 0.06826046,
       0.05988327, 0.04238394, 0.05229795, 0.05715653])}

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2469
{'Expert ID': 'Expert 3', 'Weights': array([0.16651044, 0.05906558, 0.03254317, 0.06868625, 0.06471957,
       0.06471787, 0.07125901, 0.09383704, 0.0792923 , 0.07771352,
       0.07072391, 0.0459399 , 0.0459399 , 0.05905154])}

CR: 0.6528 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2245
{'Expert ID': 'Expert 

In [44]:
soc_cr_psi_values = {
    'Expert 1': {'CR': 1.0, 'PSI': 0.1988},
    'Expert 2': {'CR': 1.0, 'PSI': 0.1836},
    'Expert 3': {'CR': 1.0, 'PSI': 0.2469},
    'Expert 4': {'CR': 0.6528, 'PSI': 0.2245},
    'Expert 5': {'CR': 1.0, 'PSI': 0.0638},
    'Expert 6': {'CR': 1.0, 'PSI': 0.1244},
    'Expert 7': {'CR': 1.0, 'PSI': 0.2113},
    'Expert 8': {'CR': 0.5, 'PSI': 0.0572},
    'Expert 9': {'CR': 1.0, 'PSI': 0.0417}
}

add_cr_psi_from_dict(soc_expert_results, soc_cr_psi_values)
soc_expert_results

[{'Expert ID': 'Expert 1',
  'Weights': array([0.07175668, 0.06421034, 0.07571554, 0.05734989, 0.05735006,
         0.05638111, 0.25998835, 0.0717893 , 0.05079942, 0.0482347 ,
         0.03070835, 0.04782889, 0.05522773, 0.05265963]),
  'Consistency Ratio': 1.0,
  'PSI': 0.1988},
 {'Expert ID': 'Expert 2',
  'Weights': array([0.06116048, 0.03225975, 0.07205954, 0.05049694, 0.05932126,
         0.06030516, 0.06848955, 0.0710692 , 0.24485596, 0.06826046,
         0.05988327, 0.04238394, 0.05229795, 0.05715653]),
  'Consistency Ratio': 1.0,
  'PSI': 0.1836},
 {'Expert ID': 'Expert 3',
  'Weights': array([0.16651044, 0.05906558, 0.03254317, 0.06868625, 0.06471957,
         0.06471787, 0.07125901, 0.09383704, 0.0792923 , 0.07771352,
         0.07072391, 0.0459399 , 0.0459399 , 0.05905154]),
  'Consistency Ratio': 1.0,
  'PSI': 0.2469},
 {'Expert ID': 'Expert 4',
  'Weights': array([0.06068504, 0.05993821, 0.03528645, 0.0718534 , 0.05996872,
         0.05862066, 0.06043605, 0.20065765, 0.060

In [45]:
soc_cr_pass = get_best_experts_by_cr(soc_expert_results, threshold=0.1)
soc_cr_pass

[{'Expert ID': 'Expert 8',
  'Weights': array([0.15732769, 0.10114861, 0.02380087, 0.10157447, 0.10012717,
         0.10027202, 0.04244763, 0.04285505, 0.04290755, 0.03064525,
         0.01112418, 0.10164709, 0.04290755, 0.10121488]),
  'Consistency Ratio': 0.5,
  'PSI': 0.0572}]

In [46]:
soc_aggregated_weights = aggregate_expert_weights(soc_cr_pass, aggregation_method='arithmetic_mean')
soc_aggregated_weights

array([0.15732769, 0.10114861, 0.02380087, 0.10157447, 0.10012717,
       0.10027202, 0.04244763, 0.04285505, 0.04290755, 0.03064525,
       0.01112418, 0.10164709, 0.04290755, 0.10121488])

### **Accessibility Issues**

In [30]:
df_acc = pd.read_csv('bwm_dataset/bwm_acc.csv')
df_acc.head()

,Expert,BO. Hesitance to approach authorities due to negative experiences,BO. Limited access to information about rights and services available to them,BO. Fear of retaliation,BO. Personal and practical constraints,BO. Lack of robust support network,BO. Inaccessibility to services due to poverty,BO. Lack of financial support for indigent victims,BO. Misconception about legal costs,BO. Perceived powerlessness among victim-survivors,BO. Resignation to one's fate,BO. Insensitive frontliners,OW. Hesitance to approach authorities due to negative experiences,OW. Limited access to information about rights and services available to them,OW. Fear of retaliation,OW. Personal and practical constraints,OW. Lack of robust support network,OW. Inaccessibility to services due to poverty,OW. Lack of financial support for indigent victims,OW. Misconception about legal costs,OW. Perceived powerlessness among victim-survivors,OW. Resignation to one's fate,OW. Insensitive frontliners
0,Expert 1,7,7,6,6,7,1,7,7,7,7,8,7,7,7,6,7,8,7,1,7,7,8
1,Expert 2,6,1,6,6,6,7,7,7,6,6,7,5,6,5,6,5,6,6,7,6,1,7
2,Expert 3,3,1,5,4,7,4,7,5,6,3,7,4,5,3,5,3,3,3,5,7,1,7
3,Expert 4,7,7,9,7,8,8,8,5,1,8,5,6,7,6,4,4,7,7,1,7,5,5
4,Expert 5,1,1,1,5,5,1,1,1,1,1,7,5,5,5,5,5,7,5,1,7,5,5


In [31]:
acc_expert_results = []

for expert in df_acc['Expert'].unique():
    expert_result = run_bwm_for_expert(df_acc, expert, 11)
    print(expert_result)
    print()
    acc_expert_results.append(expert_result)

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2736
{'Expert ID': 'Expert 1', 'Weights': array([0.0760797 , 0.0760797 , 0.08869131, 0.08851781, 0.07491627,
       0.2589917 , 0.07547738, 0.042517  , 0.0760797 , 0.0760797 ,
       0.06656974])}

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1502
{'Expert ID': 'Expert 2', 'Weights': array([0.08149741, 0.338841  , 0.05502992, 0.08124186, 0.05540212,
       0.06984464, 0.06983717, 0.06986412, 0.07713879, 0.03143885,
       0.06986412])}

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.203
{'Expert ID': 'Expert 3', 'Weights': array([0.08940864, 0.24670249, 0.08857727, 0.09410504, 0.06362484,
       0.08599088, 0.0642495 , 0.0899493 , 0.07495775, 0.03818479,
       0.0642495 ])}

CR: 0.6528 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1763
{'Expert ID': 'Expert 4', 'Weights': array([0.07473348, 0.0660861 , 0.05535159, 0.07416806, 0.05704513,
       0.06386559, 0.065387

In [47]:
acc_cr_psi_values = {
    'Expert 1': {'CR': 1.0, 'PSI': 0.2736},
    'Expert 2': {'CR': 1.0, 'PSI': 0.1502},
    'Expert 3': {'CR': 1.0, 'PSI': 0.203},
    'Expert 4': {'CR': 0.6528, 'PSI': 0.1763},
    'Expert 5': {'CR': 0.6667, 'PSI': 0.1267},
    'Expert 6': {'CR': 1.0, 'PSI': 0.1517},
    'Expert 7': {'CR': 0.6667, 'PSI': 0.2145},
    'Expert 8': {'CR': 0.6667, 'PSI': 0.0723},
    'Expert 9': {'CR': 0.5556, 'PSI': 0.064}
}

add_cr_psi_from_dict(acc_expert_results, acc_cr_psi_values)
acc_expert_results

[{'Expert ID': 'Expert 1',
  'Weights': array([0.0760797 , 0.0760797 , 0.08869131, 0.08851781, 0.07491627,
         0.2589917 , 0.07547738, 0.042517  , 0.0760797 , 0.0760797 ,
         0.06656974]),
  'Consistency Ratio': 1.0,
  'PSI': 0.2736},
 {'Expert ID': 'Expert 2',
  'Weights': array([0.08149741, 0.338841  , 0.05502992, 0.08124186, 0.05540212,
         0.06984464, 0.06983717, 0.06986412, 0.07713879, 0.03143885,
         0.06986412]),
  'Consistency Ratio': 1.0,
  'PSI': 0.1502},
 {'Expert ID': 'Expert 3',
  'Weights': array([0.08940864, 0.24670249, 0.08857727, 0.09410504, 0.06362484,
         0.08599088, 0.0642495 , 0.0899493 , 0.07495775, 0.03818479,
         0.0642495 ]),
  'Consistency Ratio': 1.0,
  'PSI': 0.203},
 {'Expert ID': 'Expert 4',
  'Weights': array([0.07473348, 0.0660861 , 0.05535159, 0.07416806, 0.05704513,
         0.06386559, 0.0653871 , 0.03429254, 0.34682976, 0.06538447,
         0.09685618]),
  'Consistency Ratio': 0.6528,
  'PSI': 0.1763},
 {'Expert ID': 'Ex

In [48]:
acc_cr_pass = get_best_experts_by_cr(acc_expert_results, threshold=0.1)
acc_cr_pass

[{'Expert ID': 'Expert 9',
  'Weights': array([0.02687517, 0.17900954, 0.11594506, 0.11807272, 0.03360015,
         0.11923756, 0.11638334, 0.12269675, 0.12096686, 0.01394808,
         0.03326478]),
  'Consistency Ratio': 0.5556,
  'PSI': 0.064}]

In [49]:
acc_aggregated_weights = aggregate_expert_weights(acc_cr_pass, aggregation_method='arithmetic_mean')
acc_aggregated_weights

array([0.02687517, 0.17900954, 0.11594506, 0.11807272, 0.03360015,
       0.11923756, 0.11638334, 0.12269675, 0.12096686, 0.01394808,
       0.03326478])

### **Political Factors**

In [33]:
df_pol = pd.read_csv('bwm_dataset/bwm_pol.csv')
df_pol.head()

,Expert,BO. Weak and/or lack of enforcement of gender mainstreaming efforts,BO. Differing priorities within barangays,BO. Weak political commitment to addressing VAW at various government levels,BO. Limited representation of gender equality advocates in leadership positions,BO. Inconsistent transition of officers in between terms of differing political figures,BO. Discontinuation of projects due to politicking,BO. Decision is influenced by politicians,BO. Political figures choose officers based on their personal preference over merit,BO. Lack of accountability mechanisms for power misuse,OW. Weak and/or lack of enforcement of gender mainstreaming efforts,OW. Differing priorities within barangays,OW. Weak political commitment to addressing VAW at various government levels,OW. Limited representation of gender equality advocates in leadership positions,OW. Inconsistent transition of officers in between terms of differing political figures,OW. Discontinuation of projects due to politicking,OW. Decision is influenced by politicians,OW. Political figures choose officers based on their personal preference over merit,OW. Lack of accountability mechanisms for power misuse
0,Expert 1,7,3,8,8,1,8,8,8,7,7,1,8,8,8,8,8,8,7
1,Expert 2,7,8,7,6,1,7,7,8,7,6,6,7,6,7,6,8,7,1
2,Expert 3,5,3,3,5,1,3,3,5,6,7,5,6,8,5,5,1,3,2
3,Expert 4,8,8,8,8,1,8,6,7,9,7,1,5,5,6,6,6,5,7
4,Expert 5,5,5,7,1,1,1,1,1,1,5,5,1,5,5,5,5,5,7


In [34]:
pol_expert_results = []

for expert in df_pol['Expert'].unique():
    expert_result = run_bwm_for_expert(df_pol, expert, 9)
    print(expert_result)
    print()
    pol_expert_results.append(expert_result)

CR: 1.0 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2551
{'Expert ID': 'Expert 1', 'Weights': array([0.09080127, 0.04179455, 0.07934982, 0.0794522 , 0.38050263,
       0.07923778, 0.07945243, 0.07942025, 0.08998908])}

CR: 0.8571 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.1845
{'Expert ID': 'Expert 2', 'Weights': array([0.08562746, 0.07297195, 0.07538183, 0.09086873, 0.41595253,
       0.07034451, 0.08385236, 0.07186567, 0.03313496])}

CR: 1.1333 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2038
{'Expert ID': 'Expert 3', 'Weights': array([0.09078162, 0.12264256, 0.12381743, 0.09023748, 0.25307351,
       0.11631155, 0.03676653, 0.09010033, 0.07626899])}

CR: 0.75 (The Consistency Level is Not Acceptable)
Epsilon Value: 0.2125
{'Expert ID': 'Expert 4', 'Weights': array([0.07259906, 0.03929342, 0.07604797, 0.07537849, 0.40438848,
       0.07964677, 0.10273803, 0.08783285, 0.06207492])}

CR: 0.4286 (The Consistency Level is Not Acceptable)
Epsi

In [50]:
pol_cr_psi_values = {
    'Expert 1': {'CR': 1.0, 'PSI': 0.2551},
    'Expert 2': {'CR': 0.8571, 'PSI': 0.1845},
    'Expert 3': {'CR': 1.1333, 'PSI': 0.2038},
    'Expert 4': {'CR': 0.75, 'PSI': 0.2125},
    'Expert 5': {'CR': 0.4286, 'PSI': 0.0735},
    'Expert 6': {'CR': 0.6667, 'PSI': 0.1636},
    'Expert 7': {'CR': 1.0, 'PSI': 0.2209},
    'Expert 8': {'CR': 0.5556, 'PSI': 0.2348},
    'Expert 9': {'CR': 0.5556, 'PSI': 0.1058}
}

add_cr_psi_from_dict(pol_expert_results, pol_cr_psi_values)
pol_expert_results

[{'Expert ID': 'Expert 1',
  'Weights': array([0.09080127, 0.04179455, 0.07934982, 0.0794522 , 0.38050263,
         0.07923778, 0.07945243, 0.07942025, 0.08998908]),
  'Consistency Ratio': 1.0,
  'PSI': 0.2551},
 {'Expert ID': 'Expert 2',
  'Weights': array([0.08562746, 0.07297195, 0.07538183, 0.09086873, 0.41595253,
         0.07034451, 0.08385236, 0.07186567, 0.03313496]),
  'Consistency Ratio': 0.8571,
  'PSI': 0.1845},
 {'Expert ID': 'Expert 3',
  'Weights': array([0.09078162, 0.12264256, 0.12381743, 0.09023748, 0.25307351,
         0.11631155, 0.03676653, 0.09010033, 0.07626899]),
  'Consistency Ratio': 1.1333,
  'PSI': 0.2038},
 {'Expert ID': 'Expert 4',
  'Weights': array([0.07259906, 0.03929342, 0.07604797, 0.07537849, 0.40438848,
         0.07964677, 0.10273803, 0.08783285, 0.06207492]),
  'Consistency Ratio': 0.75,
  'PSI': 0.2125},
 {'Expert ID': 'Expert 5',
  'Weights': array([0.05515068, 0.05515068, 0.02573698, 0.20221917, 0.13277713,
         0.13277625, 0.13132221, 0.131

In [51]:
pol_cr_pass = get_best_experts_by_cr(pol_expert_results, threshold=0.1)
pol_cr_pass

[{'Expert ID': 'Expert 5',
  'Weights': array([0.05515068, 0.05515068, 0.02573698, 0.20221917, 0.13277713,
         0.13277625, 0.13132221, 0.13111419, 0.13375272]),
  'Consistency Ratio': 0.4286,
  'PSI': 0.0735}]

In [52]:
pol_aggregated_weights = aggregate_expert_weights(pol_cr_pass, aggregation_method='arithmetic_mean')
pol_aggregated_weights

array([0.05515068, 0.05515068, 0.02573698, 0.20221917, 0.13277713,
       0.13277625, 0.13132221, 0.13111419, 0.13375272])